In [1]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import gc
import math
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s

def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo



In [2]:
# select G type stars

data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction.csv',sep=',')


In [6]:
#M
count=0
for i in range(0,len(data)):
    if data['teff_b2018'][i]>=2400 and data['teff_b2018'][i]<3700:
        count+=1

print(count)

#K
count=0
for i in range(0,len(data)):
    if data['teff_b2018'][i]>=3700 and data['teff_b2018'][i]<5200:
        count+=1

print(count)

#G
count=0
for i in range(0,len(data)):
    if data['teff_b2018'][i]>=5200 and data['teff_b2018'][i]<6000:
        count+=1

print(count)

#K
count=0
for i in range(0,len(data)):
    if data['teff_b2018'][i]>=6000 and data['teff_b2018'][i]<7500:
        count+=1

print(count)

1801
16489
46281
47667


In [3]:
data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction.csv',sep=',')


In [5]:
koi_table.columns

Index(['kepid', 'ra', 'dec', 'cdpp3', 'cdpp6', 'cdpp12', 'std_vra', 'std_vdec',
       'mean_vra', 'mean_vdec', 'sigma_vra', 'sigma_vdec', 'kepmag', 'teff',
       'logg', 'radius', 'feh', 'mass', 'dens', 'dist', 'av', 'jmag', 'hmag',
       'kmag', 'ra_gaia', 'dec_gaia', 'parallax', 'parallax_error', 'pmra',
       'pmra_error', 'pmdec', 'pmdec_error', 'phot_g_mean_mag', 'teff_b2018',
       'teffe_b2018', 'rad_b2018', 'radep_b2018', 'radem_b2018', 'koi_period',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_model_snr'],
      dtype='object')

In [4]:
data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction.csv',sep=',')

with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['teff_b2018'][i]>=5200 and data['teff_b2018'][i]<6000:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })


In [9]:
data=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction.csv',sep=',')
print(data.columns)

Index(['kepid', 'ra', 'dec', 'cdpp3', 'cdpp6', 'cdpp12', 'std_vra', 'std_vdec',
       'mean_vra', 'mean_vdec', 'sigma_vra', 'sigma_vdec', 'kepmag', 'teff',
       'logg', 'radius', 'feh', 'mass', 'dens', 'dist', 'av', 'jmag', 'hmag',
       'kmag', 'ra_gaia', 'dec_gaia', 'parallax', 'parallax_error', 'pmra',
       'pmra_error', 'pmdec', 'pmdec_error', 'phot_g_mean_mag', 'teff_b2018',
       'teffe_b2018', 'rad_b2018', 'radep_b2018', 'radem_b2018', 'koi_period',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_model_snr'],
      dtype='object')


In [3]:
data=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_new.csv',sep=',')

with open('data/kepler_koi_q17_add_dispersions_reduction_reduction_G_new.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
       'koi_period',
       'koi_duration',
       'koi_depth',
       'koi_prad',
       'koi_model_snr',
        'koi_disposition',
        'koi_pdisposition',
        'koi_teq',       
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['teff_b2018'][i]>=5200 and data['teff_b2018'][i]<6000:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            'koi_period':data['koi_period'][i],
                            'koi_duration':data['koi_period'][i],
                            'koi_depth':data['koi_depth'][i],
                            'koi_prad':data['koi_prad'][i],
                            'koi_model_snr':data['koi_model_snr'][i],
                            'koi_disposition':data['koi_disposition'][i],
                            'koi_pdisposition':data['koi_pdisposition'][i],
                            'koi_teq':data['koi_teq'][i],                             
                             
                            })


In [2]:
data=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_G_new.csv',sep=',')

with open('data/kepler_koi_q17_add_dispersions_reduction_reduction_G_2sigma_hot.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
       'koi_period',
       'koi_duration',
       'koi_depth',
       'koi_prad',
       'koi_model_snr',
        'koi_disposition',
        'koi_pdisposition',
        'koi_teq',       
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]>=2 or data['sigma_vdec'][i]>=2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            'koi_period':data['koi_period'][i],
                            'koi_duration':data['koi_period'][i],
                            'koi_depth':data['koi_depth'][i],
                            'koi_prad':data['koi_prad'][i],
                            'koi_model_snr':data['koi_model_snr'][i],
                            'koi_disposition':data['koi_disposition'][i],
                            'koi_pdisposition':data['koi_pdisposition'][i],
                            'koi_teq':data['koi_teq'][i],                             
                             
                            })

with open('data/kepler_koi_q17_add_dispersions_reduction_reduction_G_2sigma_cool.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
       'koi_period',
       'koi_duration',
       'koi_depth',
       'koi_prad',
       'koi_model_snr',
        'koi_disposition',
        'koi_pdisposition',
        'koi_teq',       
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]<2 and data['sigma_vdec'][i]<2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            'koi_period':data['koi_period'][i],
                            'koi_duration':data['koi_period'][i],
                            'koi_depth':data['koi_depth'][i],
                            'koi_prad':data['koi_prad'][i],
                            'koi_model_snr':data['koi_model_snr'][i],
                            'koi_disposition':data['koi_disposition'][i],
                            'koi_pdisposition':data['koi_pdisposition'][i],
                            'koi_teq':data['koi_teq'][i],                             
                             
                            })


In [3]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_G.csv',sep=',')

In [4]:
def N_s(Rp,P,i):
    os.system('mkdir result/N_G_2020/')
    N_s_temp=0
    with open('result/N_G_2020/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(stellar_table)):
            a_temp=a(stellar_table['mass'][i],P)
            tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
            delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
            n_temp=n(stellar_table['t_obs'][i],P)
            x=[3,6,12]
            y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(stellar_table['t_obs'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def N_s2(Rp,P):
#    os.system('mkdir result/N_s/')
    N_s_temp=0
#    with open('result/N_s/'+str(kepid)+'.csv','w') as f:
#        s=['SNR',
#           'fn',
#           'feff',]
#        writer=csv.DictWriter(f,fieldnames=s)
#        writer.writeheader()        
    for i in range(0,len(stellar_table)):
        a_temp=a(stellar_table['mass'][i],P)
        tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
        delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
        n_temp=n(stellar_table['t_obs'][i],P)
        x=[3,6,12]
        y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
        popt, pcov = curve_fit(sigma_s, x, y)
        sigma_LC=popt[0]
        cdppindex=popt[1]
        sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
        SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
        fn_temp=f_n(stellar_table['t_obs'][i],P)
        feff_temp=f_eff(SNR_temp)
#        writer.writerow({'SNR':SNR_temp,
#                         'fn':fn_temp,
#                         'feff':feff_temp,})                             
        N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def f_occurrence_dot(Rp,P,Ms,Rs,e,i):
    N_s_temp=N_s(Rp,P,i)
    a_temp=a(Ms,P)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    return f_occurrence_temp
    

def f_occurrence_cell(Rp_low,Rp_high,P_low,P_high,f_occurrence_everyplanet): # the log base of Rp and P
    count=0
    for i in range(0,len(koi_table)):
        if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high:
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(koi_table)):
            if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high:
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                
                f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,0
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        fgeo_temp=0.1
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,0



In [5]:
#print(Ns_temp)
f_occurrence_everyplanet=[]
for i in range(983,len(koi_table)):
    P=koi_table['koi_period'][i]
    Rp=koi_table['koi_prad'][i]
    Ms=koi_table['mass'][i]
    Rs=koi_table['rad_b2018'][i]
    a_temp=a(Ms,P)
    N_s_temp=N_s(Rp,P,i)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    f_occurrence_everyplanet.append(f_occurrence_temp)
    print(i,f_occurrence_temp,koi_table['kepid'][i])

983 0.004663143264566376 4851239
984 6.20401164882774e-05 8879915
985 0.0006614501017199414 9837685
986 0.002254303429299237 11805075
987 0.00031022847584009905 4851356
988 0.0012418065356617897 9401997
989 0.011762034078075766 12554485
990 0.0004903720517571726 5736801
991 7.230712220096915e-05 11709423
992 0.0005693091298052944 8884274
993 0.00021721471577012129 12554634
994 0.0005198370165693182 11411639
995 8.880009131038752e-05 12004679
996 0.0006667516836668471 7219906
997 0.00019637834501402623 9140402
998 0.00036880655567284563 12004680
999 0.0007895664866367189 5021899
1000 0.00066588896466877 8415745
1001 0.0003030426690234439 9597806
1002 0.0007498192465605423 10934674
1003 0.01230494759850779 11356602
1004 0.0002592191811877013 1431122
1005 0.00017520241079487854 3348082
1006 0.0007354054584176138 3348082
1007 0.00036672518301172127 3348082
1008 0.00026679617439980604 4077526
1009 0.0004362693353719503 4077526
1010 0.0010890163138320614 4077526
1011 0.0005657302798933869 40

1215 4.883380410893432e-05 10554967
1216 0.00021707669127848743 6384840
1217 0.00276546563454973 5471688
1218 0.00021083060769587825 10554999
1219 0.00034993125999008355 10554999
1220 0.0050267057032945366 10917433
1221 0.0012710052856713123 8222395
1222 0.0001073528132615411 11904704
1223 0.0009548016858677474 7103919
1224 0.00011753693286483033 9777251
1225 0.00018019446174327464 10801951
1226 0.00024354094111175107 8312222
1227 0.00020221020399184233 9466668
1228 0.00029525243248026853 9466668
1229 0.00012674146395759872 9466668
1230 0.00044110518314100693 9466668
1231 0.0003674944536529878 7708215
1232 0.0009187886482441997 6351097
1233 0.0008132148168700031 2439243
1234 0.00046233482314172287 5941160
1235 0.00039934762937062266 5941160
1236 0.0016443525954828182 4813750
1237 6.823524877114245e-05 8645191
1238 0.00024292029709440703 12159249
1239 0.00027228316771480006 4932442
1240 0.0029015656298523587 3854101
1241 0.0004103747989273019 11520459
1242 0.00014049143087713914 3328027

1448 0.0002650807588072007 3957082
1449 0.0001450861977089963 11858748
1450 0.0009302442405537854 3559935
1451 0.0016824181861019303 8231667
1452 0.0003963366211944076 8326342
1453 0.0009254147740811637 8573193
1454 0.00170482412495439 10965008
1455 6.516847930103314e-05 5895588
1456 0.08257892037610687 8748435
1457 0.0001300611458390539 9490653
1458 9.191193668710708e-05 4990977
1459 0.0005293089395342303 8587439
1460 0.0006251745930143807 9714550
1461 0.00035518066252396925 12120307
1462 0.0006012872006932552 12120307
1463 0.00044603337448334866 12120307
1464 0.00012006384166479068 3834360
1465 6.150569212738615e-05 10785538
1466 7.250218026227178e-05 4367544
1467 0.0006372261033814785 8258160
1468 0.0016946297918800954 5956342
1469 0.0003503591038374144 5956342
1470 0.0011843630424682935 5956342
1471 0.0005924020448719612 5956342
1472 0.001366457415440683 5005121
1473 3.597875001509396e-05 11391667
1474 0.0013271753906815337 4366323
1475 0.00014708792673347095 3427776
1476 0.0005467

1681 0.012209100811085354 8692861
1682 0.0004059721275441652 8692861
1683 0.0006479125368553038 11389771
1684 0.0003521382681961117 11389771
1685 0.00019737457612409174 11389771
1686 0.00011702349171314544 7849854
1687 0.0006207907528291896 3323887
1688 0.00019467499707729848 5456979
1689 0.0009542281608059408 9390228
1690 0.00038741763587725806 2557816
1691 0.0003311972568771627 8564587
1692 0.0006166470399853418 8564587
1693 6.186227429911222e-05 6590362
1694 0.00023249240995189954 4750341
1695 0.0010264823614220286 9085563
1696 0.0006645963255980589 2164169
1697 0.00022837714802377335 7604328
1698 0.0003102667012722587 7604328
1699 0.00013204964444239978 5195172
1700 9.598248621001073e-05 7200225
1701 0.0003052417992240951 11337372
1702 0.001939658796791301 11337372
1703 0.000905936573038304 5126182
1704 5.687525005157855e-05 2558363
1705 0.0007323430588918598 4248120
1706 0.002787832161207505 3832474
1707 0.0014674261256452136 3832474
1708 0.0009601093268382618 11288492
1709 0.0008

1914 0.0003102264030226212 7841925
1915 0.011417643383499662 8625732
1916 0.00017220664342704173 5471566
1917 0.0018390709195336668 8167504
1918 0.00017419635342297957 5303551
1919 0.001085575263744224 8625821
1920 0.0005338407878390693 11962284
1921 0.0002343351735311507 2708420
1922 0.00028946343295812573 8625925
1923 0.00010125944859070016 6129524
1924 0.0001326490992125618 11560037
1925 0.00018388590784280867 7287415
1926 0.0031619437301940363 7768451
1927 0.0019518204310930304 10275805
1928 8.444790616330918e-05 9656543
1929 0.0006101578036806856 7620413
1930 0.00015998552388003781 7620413
1931 0.09825366276254652 3629119
1932 0.0015183707958911082 12208631
1933 0.00010339338765478442 12208631
1934 0.001448470185968299 3448130
1935 0.0003239488529994367 12505076
1936 0.0006538580574760564 4915638
1937 0.0002628286281117945 6187508
1938 0.0003043485963528543 7987749
1939 22.161498033947577 8873873
1940 0.0008186927950710122 4175630
1941 0.00015805943043803482 5467124
1942 0.0002461

2147 0.00041326476785308243 8396288
2148 0.0003776195033490957 9269281
2149 0.01588808520090339 11124353
2150 0.0006081718862934974 9655711
2151 0.0004323007321293942 9655711
2152 0.0004177475682671748 11554435
2153 0.0004144540128727458 5687700
2154 0.001211538302310094 6678383
2155 0.00042240678063956943 6678383
2156 0.0007113990062398656 6678383
2157 0.00016221242415686578 2696205
2158 0.0003185628472551088 8364115
2159 0.0010407662276914285 11507101
2160 0.0005875387621978812 10545066
2161 0.0001509294949515435 11073351
2162 0.00018437667226901312 11072016
2163 0.0008308962664611761 9540675
2164 0.0006024931358041659 8396405
2165 0.00114662517101046 4946581
2166 0.0003576988747417241 7090524


In [8]:
# select G type stars

stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_G.csv',sep=',')


f_occurrence_everyplanet=[]

from subprocess import getoutput
filelist=getoutput('ls result/N_G_2020/*.csv')
filename=[]
for line in filelist.split('\n'):
    filename.append(line)
    

    
with open('result/occurrence_rate_G_2020.csv','w') as f:
    s=['i',
       'kepid',
       'occurrence_rate',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(filename)):
        koi_id=int(filename[i][16:-4])
        P=koi_table['koi_period'][koi_id]
        Rp=koi_table['koi_prad'][koi_id]
        Ms=koi_table['mass'][koi_id]
        Rs=koi_table['rad_b2018'][koi_id]
        a_temp=a(Ms,P)
        N_inf=pd.read_csv(filename[i],sep=',')
        fn=N_inf['fn']
        feff=N_inf['feff']
        N_s_temp=0
        for j in range(0,len(fn)):
            N_s_temp+=fn[j]*feff[j]
        
        del N_inf,fn,feff
        gc.collect()
        N_s_temp=round(N_s_temp)
        fgeo_temp=f_geo(Rp,Rs,a_temp,e)
        f_occurrence_temp=1./(fgeo_temp*N_s_temp)
        f_occurrence_everyplanet.append(f_occurrence_temp)
        print(koi_id,f_occurrence_temp,koi_table['kepid'][koi_id],len(koi_table))
        writer.writerow({'i':koi_id,
                         'kepid':koi_table['kepid'][koi_id],
                         'occurrence_rate':f_occurrence_temp,})
 

0 0.003141013765739711 8182107 2167
1000 0.00066588896466877 8415745 2167
1001 0.0003030426690234439 9597806 2167
1002 0.0007498192465605423 10934674 2167
1003 0.01230494759850779 11356602 2167
1004 0.0002592191811877013 1431122 2167
1005 0.00017520241079487854 3348082 2167
1006 0.0007354054584176138 3348082 2167
1007 0.00036672518301172127 3348082 2167
1008 0.00026679617439980604 4077526 2167
1009 0.0004362693353719503 4077526 2167
100 0.0024071571111501863 10053138 2167
1010 0.0010890163138320614 4077526 2167
1011 0.0005657302798933869 4077526 2167
1012 8.017098060424003e-05 6265792 2167
1013 0.00133396623710603 6265961 2167
1014 0.0005452802396314029 4827723 2167
1015 0.0071202724726162426 9079587 2167
1016 0.00013653419478778798 3348285 2167
1017 0.00019968911707073355 3765692 2167
1018 5.7488692683506436e-05 7220322 2167
1019 0.0026557013661919547 5737869 2167
101 6.815975091664834e-05 9574614 2167
1020 0.0012226192000462232 4077901 2167
1021 0.005196806106559938 10425070 2167
102

1184 0.0012580854498303116 8609450 2167
1185 9.737549066303327e-05 7129465 2167
1186 0.0006323446587072505 11612280 2167
1187 0.0006013965206777483 8043638 2167
1188 0.0004462288953969663 2571238 2167
1189 0.0003940641026828098 7703882 2167
118 0.0008885069819674908 7285757 2167
1190 0.00041792142723684483 10215422 2167
1191 0.00045522138744899445 6849310 2167
1192 0.00025941521751563976 6849310 2167
1193 0.000745615378443487 6849310 2167
1194 0.0008592434788638012 4932348 2167
1195 0.00015232641471156938 7703910 2167
1196 0.44035615549638546 8311864 2167
1197 0.0005388157090040991 9664142 2167
1198 0.006172850315372777 3003992 2167
1199 0.025424168046814313 3003992 2167
119 0.0009582889798800497 7285757 2167
11 0.00011961245683156352 9777290 2167
1200 0.0035028618550570424 7703955 2167
1201 0.0017345366884930187 7703955 2167
1202 0.0020852493250321153 7353970 2167
1203 0.0002216879954545301 7376490 2167
1204 0.0005119588632011987 2438513 2167
1205 0.0005285145324657677 2438513 2167
12

1369 0.0009584363489881238 10004738 2167
136 0.0009955296799749917 12253769 2167
1370 0.0023768290084875436 10004738 2167
1371 0.00020346853164573947 5775129 2167
1372 0.00039092340501049313 4989057 2167
1373 0.00013423987296853383 9818381 2167
1374 0.0004120468509411402 9047416 2167
1375 0.00018285252302416134 12110942 2167
1376 0.0003429543785809555 8836888 2167
1377 8.655032355483579e-05 7605093 2167
1378 0.00022425848044500157 11138155 2167
1379 0.00011208380965006205 10148521 2167
137 0.00018629791627046814 3851130 2167
1380 0.00019990469101085475 8255887 2167
1381 7.050303809180549e-05 5991070 2167
1382 0.0001242514707825415 9959492 2167
1383 0.003588201321010507 10733174 2167
1384 0.0001809542755747094 3324986 2167
1385 0.0004910021668987383 9959545 2167
1386 0.00017579260972123411 8324518 2167
1387 4.51273574011372e-05 10613718 2167
1388 0.00056384387573948 12456601 2167
1389 0.0010323322805754604 10864531 2167
138 0.0014822728534672036 7286173 2167
1390 0.001961324251397672 10

1552 0.0009056747494693394 7847927 2167
1553 0.00020384068776147519 5952448 2167
1554 0.0005131045579932274 10191036 2167
1555 0.0010185758527536779 11137061 2167
1556 0.0001864119379894662 11502867 2167
1557 0.00017396254607768917 2718778 2167
1558 4.549463524621285e-05 5476671 2167
1559 4.5396590428382404e-05 5476671 2167
155 5.5896872020556616e-05 11444952 2167
1560 0.00044664322478276763 5372966 2167
1561 0.0009699024257432479 6587105 2167
1562 0.0012745668057533176 10538176 2167
1563 0.0005999907962543092 10538176 2167
1564 0.0002018974917163566 6949550 2167
1565 0.0003828051495867776 9823457 2167
1566 0.0010802913623116084 9823457 2167
1567 4.186579876798513e-05 11076176 2167
1568 0.00047763971995821785 8229696 2167
1569 0.0003278625230086882 6664842 2167
156 0.002060535866632036 6289650 2167
1570 0.0006178634788530013 5456319 2167
1571 0.0020707986652793936 4841374 2167
1572 0.0023417820381682946 7131760 2167
1573 0.008072005630154973 12117215 2167
1574 0.00029722543011628284 88

1737 0.00045789529343889107 7269974 2167
1738 0.00022288102096908852 7269974 2167
1739 0.0011742837834217439 8480582 2167
173 0.014295167564423571 5305225 2167
1740 0.0006130813869803978 6035124 2167
1741 0.00014068220688596148 9705459 2167
1742 0.0006918451547978055 8760040 2167
1743 0.0004019249968581956 8760040 2167
1744 0.010389333986692987 9075882 2167
1745 0.0006007815457304929 5470739 2167
1746 3.386129352770125e-05 5636648 2167
1747 0.00021402643359796482 2849805 2167
1748 0.000884042333431967 10798838 2167
1749 0.0010926843679436005 11449844 2167
174 0.00013783107952160468 5305240 2167
1750 0.0011365931052040536 5470851 2167
1751 0.0016066789381410569 8345172 2167
1752 0.009858095901727645 4768756 2167
1753 0.002425279238054339 10528068 2167
1754 0.0011794320584868924 11253827 2167
1755 0.001900652533265625 11253827 2167
1756 0.0007334879827600256 7750419 2167
1757 7.725108701873501e-05 6690836 2167
1758 0.0016265132471481858 12403119 2167
1759 0.0001817988001438191 8709688 21

...

2105 8.447841386324906e-05 11752632 2167
2106 0.0002470661593965902 6690082 2167
2107 0.00014916819162466867 6690082 2167
2108 0.0002297616735136593 4764774 2167
2109 0.00023496422097210525 5181817 2167
210 0.0005711596542972594 3531558 2167
2110 0.0002809918368188534 9020114 2167
2111 0.0002865897830517809 12071754 2167
2112 0.0003780464109831354 11752908 2167
2113 6.74024065620436e-05 5716763 2167
2114 0.0012514732019979725 7102316 2167
2115 0.0016931680928899732 7102316 2167
2116 0.003880214248598373 7102316 2167
2117 0.00010470576485267411 7461307 2167
2118 8.754860398962429e-05 7461314 2167
2119 0.0001930638086494479 8263545 2167
211 0.0005002866083054495 9899577 2167
2120 0.00025415642548726 4545187 2167
2121 0.0014819124787050389 4545187 2167
2122 0.0015889386061550836 6619815 2167
2123 0.01639461175928994 8439618 2167
2124 0.00025963870863752195 6062088 2167
2125 0.00018129411698646842 6062088 2167
2126 0.0006650006802892716 6062088 2167
2127 0.00016596334804912574 8753657 2167

342 4.8102657513286284e-05 4551328 2167
343 0.00016651721505749759 9108085 2167
344 0.001438984493495863 11656918 2167
345 0.00010802433491116008 5903312 2167
346 0.00017074792347886508 7118364 2167
347 0.0005960991229087413 7691260 2167
348 8.482158876725829e-05 10341787 2167
349 0.00021906167690927572 8296467 2167
34 0.001010270891033109 3548044 2167
350 0.0002519306991574693 12602568 2167
351 9.793622930529883e-05 11401822 2167
352 7.76162980810719e-05 3113266 2167
353 9.793622930529883e-05 11401822 2167
354 0.00022994969736724315 8075618 2167
355 0.0008077631498911171 9458343 2167
356 0.012118363633865594 10586619 2167
357 7.124682634119727e-05 10341905 2167
358 0.00034286653130402497 7211221 2167
359 9.509988579442395e-05 10341917 2167
35 0.11233064863614227 3548044 2167
360 0.005660788193558202 7429240 2167
361 0.0012621121215206695 5898983 2167
362 0.023607074730693955 7664272 2167
363 0.0006147575635037824 10345478 2167
364 9.628343887583e-05 3935914 2167
365 0.0007508044283257

530 0.000999480341414688 12600735 2167
531 0.0005431042727035184 12600735 2167
532 0.00042524195586927135 7217088 2167
533 0.0007158612083440046 11718144 2167
534 0.0005223085585130181 4847534 2167
535 0.0002532314640787889 5007345 2167
536 0.00016447031780146852 4574319 2167
537 0.00011730971644530826 5036480 2167
538 0.0004934130721489945 10272389 2167
539 0.004431876882964958 7662744 2167
53 0.0028844426547493824 9773270 2167
540 0.00011251538842712358 7613620 2167
541 0.00012779035288026854 7903086 2167
542 0.0007819680861876859 6541920 2167
543 0.001105275172092961 6541920 2167
544 0.0004099412982059049 6541920 2167
545 0.0004371473576563699 6541920 2167
546 0.0006314888882237888 6541920 2167
547 0.0019898462050161004 6541920 2167
548 0.0006645240775672379 6933567 2167
549 0.00011320326457562906 3836375 2167
54 0.00017404897694288778 6525185 2167
550 0.0006840500864770775 6863998 2167
551 0.00041274280395953427 8547429 2167
552 0.00024106802325664478 11410904 2167
553 0.0001034612

719 0.00018484661359638137 9827149 2167
71 0.0018871544708911959 10600955 2167
720 0.00019920709827442199 7901948 2167
721 0.0005182952318540391 9049550 2167
722 0.0005493120770100906 6346809 2167
723 0.000983802088932638 8805348 2167
724 0.0001562183296060758 10872983 2167
725 0.00020249701907924206 10872983 2167
726 0.00038642330002913424 10872983 2167
727 0.0003365820311502827 9963524 2167
728 0.000494812542128976 9963524 2167
729 0.0007334022985010595 9963524 2167
72 0.0006565999878547367 6605493 2167
730 0.00022263315788694394 9963524 2167
731 0.00010569480016973583 6153672 2167
732 0.0005358463884226376 6846911 2167
733 0.00041191187091149445 10616679 2167
734 5.972409387180771e-05 6365321 2167
735 0.00034154679945944875 2831055 2167
736 0.0002456851434042689 2445129 2167
737 0.00023228300915093382 8180063 2167
738 7.590308361159368e-05 8260198 2167
739 0.00015121774025622309 7115200 2167
73 0.00035353500098167783 9139084 2167
740 0.00028607108747178496 7281668 2167
741 0.0002997

907 0.0018607549065537909 9394762 2167
908 0.00030703953528432907 6922203 2167
909 0.00026851229412641115 9650989 2167
90 0.0008509649064372181 9406990 2167
910 0.00019713143917315354 11359879 2167
911 0.0005925193693021724 9597058 2167
912 0.0006388982298489462 10154388 2167
913 0.00066681611089764 8174625 2167
914 0.0006180331225674575 7282168 2167
915 0.0024690396562509647 4918309 2167
916 0.002154002066361413 5217891 2167
917 0.0005753601097208954 10190075 2167
918 0.000589362698941854 11709124 2167
919 0.0013021654585550545 11709124 2167
91 0.00022808010646733703 9115800 2167
920 0.0008908626924752401 11709124 2167
921 0.0005895198510754659 11709124 2167
922 0.00022199807229896887 11709124 2167
923 0.0017715308128626974 4371172 2167
924 0.0003185435707066964 5272878 2167
925 0.00022032678710779468 6144039 2167
926 0.0006668158751299179 1995519 2167
927 0.0034867430207413514 9579641 2167
928 0.00033566760630865427 9579641 2167
929 0.00025693075643520017 9579641 2167
92 0.0003103394

In [6]:
filename

['result/N_G_2020/0.csv',
 'result/N_G_2020/1000.csv',
 'result/N_G_2020/1001.csv',
 'result/N_G_2020/1002.csv',
 'result/N_G_2020/1003.csv',
 'result/N_G_2020/1004.csv',
 'result/N_G_2020/1005.csv',
 'result/N_G_2020/1006.csv',
 'result/N_G_2020/1007.csv',
 'result/N_G_2020/1008.csv',
 'result/N_G_2020/1009.csv',
 'result/N_G_2020/100.csv',
 'result/N_G_2020/1010.csv',
 'result/N_G_2020/1011.csv',
 'result/N_G_2020/1012.csv',
 'result/N_G_2020/1013.csv',
 'result/N_G_2020/1014.csv',
 'result/N_G_2020/1015.csv',
 'result/N_G_2020/1016.csv',
 'result/N_G_2020/1017.csv',
 'result/N_G_2020/1018.csv',
 'result/N_G_2020/1019.csv',
 'result/N_G_2020/101.csv',
 'result/N_G_2020/1020.csv',
 'result/N_G_2020/1021.csv',
 'result/N_G_2020/1022.csv',
 'result/N_G_2020/1023.csv',
 'result/N_G_2020/1024.csv',
 'result/N_G_2020/1025.csv',
 'result/N_G_2020/1026.csv',
 'result/N_G_2020/1027.csv',
 'result/N_G_2020/1028.csv',
 'result/N_G_2020/1029.csv',
 'result/N_G_2020/102.csv',
 'result/N_G_2020/10

In [13]:
data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G.csv',sep=',')

with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G_hot.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]>=2 or data['sigma_vdec'][i]>=2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })

with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G_cool.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]<2 and data['sigma_vdec'][i]<2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })


In [14]:
stellar_hot=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G_hot.csv',sep=',')
stellar_cool=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_G_cool.csv',sep=',')

feh_hot=np.median(stellar_hot['feh'])
feh_cool=np.median(stellar_cool['feh'])

mass_hot=np.median(stellar_hot['mass'])
mass_cool=np.median(stellar_cool['mass'])

print(feh_hot,feh_cool,mass_hot,mass_cool)

-0.22 -0.12 0.893 0.924


In [15]:
print(len(stellar_hot),len(stellar_cool),len(data))

4310 41971 46281
